### Learning ipyrad output files

**20170517**

I want to get the hang of navigating and plotting things from the output files in ipyrad.



In [4]:
cd /mnt/hgfs/SHARED_FOLDER/Learn_iPyrad/CRAGIG_RUN1_py

/mnt/hgfs/SHARED_FOLDER/Learn_iPyrad/CRAGIG_RUN1_py


First, I can get the summary stats and paths to all the output files using the ``-r`` flag:

In [5]:
!ipyrad -p params-cragig002.txt -r


Summary stats of Assembly cragig002
------------------------------------------------
         state  reads_raw  reads_passed_filter  clusters_total  \
FG001        6    4496623              4496623           28719   
FG009        6    5828500              5828500           31799   
FG100_B      6    4191123              4191123           27072   
FG102_B      6    3757306              3757306           32437   
FG205        6    4762949              4762949           28809   
FG212        6    4163068              4163068            1568   
Q314         6    3797205              3797205           32843   
Q324         6    4203350              4203350           47166   
Q330         6    3683576              3683576           63309   
Q339         6    6057345              6057345          142327   

         clusters_hidepth  hetero_est  error_est  reads_consens  
FG001               18878    0.021602   0.002395          16813  
FG009               19670    0.021421   0.002188       

Clustering is one of the most important and sensitive steps. So it's worth understanding what's in the s3_cluster_stats.txt files. I talked to Katherine on the phone to help make sense of the headers, because the documentation is not extremely thorough (the cost of the writers making everything sleek and simple!). So, here's the file:

![img](https://github.com/nclowell/RAD_Scallops/blob/master/CRAGIG_run1/Notebooks/images_for_notebooks/ex_cluster_stats_file.png?raw=true)

I wrote a script that produces scatter plots comparing different output stats. Script lives [here](https://github.com/nclowell/RAD_Scallops/blob/master/CRAGIG_run1/Scripts/data_exploration/ipyrad_clust_stats_plots.py). It allows optional flags so you can specify which plots you want made. 

Example usage:

In [9]:
cd /mnt/hgfs/SHARED_FOLDER/Git_repo/CRAGIG_run1/Scripts/data_exploration

/mnt/hgfs/SHARED_FOLDER/Git_repo/CRAGIG_run1/Scripts/data_exploration


In [11]:
!python ipyrad_clust_stats_plots.py -h

usage: ipyrad_clust_stats_plots.py [-h] -a ASSEMBLY -i INFILE [-o OUTDIR] [-x]
                                   [-y] [-z]

Produces plots from ipyrad cluster stats output file

optional arguments:
  -h, --help            show this help message and exit
  -a ASSEMBLY, --assembly ASSEMBLY
                        Assembly name
  -i INFILE, --infile INFILE
                        Path to cluster stats output file from ipyrad
  -o OUTDIR, --outdir OUTDIR
                        Path to directory for output files
  -x                    Make scatter plot showing total number of clusters vs
                        number of clusters filtered for depth
  -y                    Make plot showing average depth with majority rule vs
                        number of clusters filtered for depth
  -z                    Make plot showing standard deviation of read depth
                        with majority rule vs number of clusters filtered for
                        depth


Plots look like this:

![img]()
![img]()
![img]()